In [ ]:
!pip install scapy


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 15.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for scapy: filename=scapy-2.5.0-py2.py3-none-any.whl size=1444327 sha256=33418bc5ddfef997469d59d733a4d069f01afb1bb466083b7e88e021e7084f8a
  Stored in directory: /root/.cache/pip/wheels/82/b7/03/8344d8cf6695624746311bc0d389e9d05535ca83c35f90241d
Successfully built scapy


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install rarfile

import rarfile

# Path to the RAR file
rar_file_path = '/content/drive/MyDrive/Caida 2007/labeled_ddostrace.to-victim.20070804 sep Fastest.rar'

# Destination directory for extraction
extracted_directory = '/content/extracted/'

# Create the destination directory if it doesn't exist
import os
os.makedirs(extracted_directory, exist_ok=True)

# Open the RAR file
with rarfile.RarFile(rar_file_path, 'r') as rar:
    # Extract all files to the destination directory
    rar.extractall(extracted_directory)

print(f"Extraction complete. Files are in: {extracted_directory}")


Extraction complete. Files are in: /content/extracted/


In [ ]:
!cd /content/extracted
!ls

ddostrace.to-victim.20070804_145436.pcap  drive  extracted  sample_data


In [ ]:
from scapy.all import *
import os
from tqdm import tqdm

def split_pcap(input_file):
    # Create a directory to store the cropped pcap files if it doesn't exist
    if not os.path.exists("cropped"):
        os.makedirs("cropped")

    # Get the base name of the input file
    base_name = os.path.splitext(os.path.basename(input_file))[0]

    # Define the time interval (5 seconds)
    interval = 1

    # Initialize variables for file index and packet count
    file_index = 1
    packet_count = 0

    # Open the pcap file in read mode
    with PcapReader(input_file) as pcap_reader:
        # Initialize tqdm progress bar
        progress_bar = tqdm(desc=f"Splitting {input_file}", unit=" packets")

        # Initialize list to store packets for the current interval
        interval_packets = []

        # Loop through each packet in the pcap file
        for packet in pcap_reader:
            # Calculate the elapsed time since the start of the capture
            elapsed_time = packet.time - interval_packets[0].time if interval_packets else 0

            # Check if elapsed time exceeds the interval
            if elapsed_time >= interval:
                # Write the interval packets to a new pcap file
                output_file = f"cropped/{base_name}_{file_index}.pcap"
                wrpcap(output_file, interval_packets)

                # Reset variables for the next interval
                interval_packets = [packet]
                file_index += 1
            else:
                # Add the packet to the current interval
                interval_packets.append(packet)

            # Increment packet count
            packet_count += 1

            # Update the progress bar
            progress_bar.update(1)

        # Close the progress bar
        progress_bar.close()

    print(f"Split {input_file} into {file_index} parts successfully.")

# Get a list of all .pcap files in the current directory
pcap_files = [f for f in os.listdir() if f.endswith('.pcap')]

# Iterate over each .pcap file and split it
for pcap_file in pcap_files:
    split_pcap(pcap_file)


Splitting ddostrace.to-victim.20070804_145436.pcap: 2450013 packets [26:56, 1516.05 packets/s]


Split ddostrace.to-victim.20070804_145436.pcap into 15 parts successfully.


In [8]:
from scapy.all import *
import pandas as pd
import os
from math import log2

# Function to calculate entropy
def calculate_entropy(data):
    entropy = 0
    total_count = len(data)
    value_counts = data.value_counts()
    for count in value_counts:
        probability = count / total_count
        entropy -= probability * log2(probability)
    return entropy

# Function to extract features from pcap file
def extract_features_from_pcap(pcap_file):
    packets = rdpcap('/content/cropped/'+pcap_file)

    # Initialize lists to store extracted features
    src_ips = []
    src_ports = []
    dst_ports = []
    packet_protocols = []
    total_packets = len(packets)

    # Extract features from each packet
    for packet in packets:
        if IP in packet:
            src_ips.append(packet[IP].src)
            if packet.haslayer(TCP) or packet.haslayer(UDP):
                src_ports.append(packet[IP].sport)
                dst_ports.append(packet[IP].dport)
            else:
                src_ports.append(None)
                dst_ports.append(None)
            packet_protocols.append(packet[IP].proto)

    # Calculate entropies
    etpSrcIP = calculate_entropy(pd.Series(src_ips))
    etpSrcP = calculate_entropy(pd.Series(src_ports))
    etpDstP = calculate_entropy(pd.Series(dst_ports))
    etpProtocol = calculate_entropy(pd.Series(packet_protocols))

    return {
        'etpSrcIP': etpSrcIP,
        'etpSrcP': etpSrcP,
        'etpDstP': etpDstP,
        'etpProtocol': etpProtocol,
        'totalPacket': total_packets
    }

# Iterate over pcap files in the directory
pcap_files = [file for file in os.listdir("/content/cropped") if file.endswith('.pcap')]
for pcap_file in pcap_files:
    print(f'Processing {pcap_file}...')
    features = extract_features_from_pcap(pcap_file)
    filename = os.path.splitext(pcap_file)[0] + '.csv'
    pd.DataFrame([features]).to_csv(filename, index=False)
    print(f'Features extracted and saved to {filename}')


Processing ddostrace.to-victim.20070804_145436_12.pcap...
Features extracted and saved to ddostrace.to-victim.20070804_145436_12.csv
Processing ddostrace.to-victim.20070804_145436_3.pcap...
Features extracted and saved to ddostrace.to-victim.20070804_145436_3.csv
Processing ddostrace.to-victim.20070804_145436_8.pcap...
Features extracted and saved to ddostrace.to-victim.20070804_145436_8.csv
Processing ddostrace.to-victim.20070804_145436_6.pcap...
Features extracted and saved to ddostrace.to-victim.20070804_145436_6.csv
Processing ddostrace.to-victim.20070804_145436_10.pcap...
Features extracted and saved to ddostrace.to-victim.20070804_145436_10.csv
Processing ddostrace.to-victim.20070804_145436_11.pcap...
Features extracted and saved to ddostrace.to-victim.20070804_145436_11.csv
Processing ddostrace.to-victim.20070804_145436_2.pcap...
Features extracted and saved to ddostrace.to-victim.20070804_145436_2.csv
Processing ddostrace.to-victim.20070804_145436_13.pcap...
Features extracted 

In [9]:
import os
import pandas as pd

def combine_csv_files():
    # Get the current directory
    current_directory = os.getcwd()

    # List all files in the current directory
    files = os.listdir(current_directory)

    # Filter out only CSV files
    csv_files = [file for file in files if file.endswith('.csv')]

    if not csv_files:
        print("No CSV files found in the current directory.")
        return

    # Read the first CSV file to get the column names
    first_file = csv_files[0]
    df_combined = pd.read_csv(first_file)
    print(df_combined)
    # Combine all CSV files
    for file in csv_files[1:]:
        df = pd.read_csv(file)
        df_combined = pd.concat([df_combined, df], ignore_index=True)

    # Write the combined dataframe to a new CSV file
    combined_filename = "combined.csv"
    df_combined.to_csv(combined_filename, index=False)

    print(f"All CSV files combined into '{combined_filename}'.")

if __name__ == "__main__":
    combine_csv_files()


    etpSrcIP   etpSrcP   etpDstP  etpProtocol  totalPacket
0  11.908531  1.158816  0.748006     0.358235       170458
All CSV files combined into 'combined.csv'.
